In [1]:
%pip install -q ipywidgets

import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive
import numpy as np

def two_point_set_transfer_function(fish_size, obs_value, tf_size, tf_full_fxn, tf_no_fxn):
  # determine if the transfer function returns full function at high observed value
  high_good = np.all((tf_full_fxn-tf_no_fxn)>0)

  interp_full_fxn_TDR = np.interp(fish_size, tf_size, tf_full_fxn)
  interp_no_fxn_TDR = np.interp(fish_size, tf_size, tf_no_fxn)
  
  if high_good:
    return np.interp(obs_value, np.array([interp_no_fxn_TDR, interp_full_fxn_TDR]), np.array([0.0,1.0]))
  else:
    return np.interp(obs_value, np.array([interp_full_fxn_TDR, interp_no_fxn_TDR]), np.array([1.0,0.0]))

def plot_contour(metric, fish_sizes, tf_levels, axes):
  n_display_levels = 50
  y_array = np.linspace(0, metric['max'], n_display_levels)
  data_matrix = np.zeros([n_display_levels, len(fish_sizes)])

  for i in range(n_display_levels):
    for j in range(len(fish_sizes)):
      data_matrix[i, j] = two_point_set_transfer_function(
      fish_sizes[j],
      y_array[i],
      metric['TF_size'],
      metric['TF_full_fcn'],
      metric['TF_no_fcn']
    )
  axes.set_xlim([20,1200])
  axes.set_ylim([0.0,metric['max']])
  axes.axhline(metric['value'], ls='--', color='hotpink')
  axes.set_ylabel(metric['title'])
  
  m = axes.contourf(fish_sizes, y_array, data_matrix, tf_levels)
  plt.colorbar(m, cax=plt.axes([0.455, 0.800, 0.01, 0.125]))

def plot_metric_line(metric_definitions, fish_sizes, axes):
  lines = {}
  for m in metric_definitions.keys():
    line_fish_size_array = np.arange(min(fish_sizes), max(fish_sizes), 1)
    lines[m] = np.zeros([len(line_fish_size_array)])
    for j in range(len(line_fish_size_array)):
      lines[m][j] = two_point_set_transfer_function(
        line_fish_size_array[j],
        metric_definitions[m]['value'],
        metric_definitions[m]['TF_size'],
        metric_definitions[m]['TF_full_fcn'],
        metric_definitions[m]['TF_no_fcn'])
        
  final_tf_line = lines['EntryHeadloss'] * \
                  np.minimum(lines['Headloss'], lines['Turbulence']) * \
                  np.minimum(lines['SlotWidth'], lines['MinDepth'])
  axes.set_ylim(0, 1)
  axes.plot(line_fish_size_array, final_tf_line)
            
def plot_metric(entry_velocity, max_slot_loss, max_ave_turb, min_slot_width, min_depth):
    fish_sizes = np.arange(20, 1201, 10)
    # Draft Transfer function inputs
    metric_definitions = {
      'EntryHeadloss': {
        'title': 'Entry Slot Velocity [ms^-1]',
        'max': 1.5,
        'value': entry_velocity, 
        'TF_size': np.array([20,1200]),
        'TF_full_fcn': np.array([1.0, 1.0]),
        'TF_no_fcn': np.array([0.3, 0.3])
        },
      'Headloss': {
        'title': 'Max Slot Loss [mm]',
        'max': 400.0,
        'value': max_slot_loss, 
        'TF_size': np.array([20, 100, 200, 700, 1200]),
        'TF_full_fcn': np.array([75, 100, 120, 165, 165]),
        'TF_no_fcn': np.array([75, 100, 120, 165, 165]) * 2
      },
      'Turbulence': {
        'title': 'Max Cell Turb. [Wm^-3]',
        'max': 150.0,
        'value': max_ave_turb, 
        'TF_size': np.array([20, 100, 200, 700, 1200]),
        'TF_full_fcn': np.array([25, 30, 50, 50, 50]),
        'TF_no_fcn': np.array([25, 30, 50, 50, 50]) * 2
      },
      'SlotWidth': {
        'title': 'Min Slot Width [mm]',
        'max': 500.0,
        'value': min_slot_width, 
        'TF_size': np.array([20, 100, 200, 700, 1200]),
        'TF_full_fcn': np.array([100, 100, 150, 250, 350]),
        'TF_no_fcn': np.array([100, 100, 150, 250, 350]) * 0.5
      },
      'MinDepth': {
        'title': 'Min Depth [m]',
        'max': 3.0,
        'value': min_depth, 
        'TF_size': np.array([20, 100, 200, 700, 1200]),
        'TF_full_fcn': np.array([0.5, 0.5, 0.75, 1.0, 1.5]),
        'TF_no_fcn': np.array([0.5, 0.5, 0.75, 1.0, 1.5]) * 0.5
      }
    }
    
    tf_levels = np.linspace(0.0, 1.0, 11)
    plt.figure(1, figsize=(16, 10))
    
    plot_contour(metric_definitions['EntryHeadloss'], fish_sizes, tf_levels, plt.axes([0.10, 0.800, 0.35, 0.125])),
    plot_contour(metric_definitions['Headloss'], fish_sizes, tf_levels, plt.axes([0.10, 0.625, 0.35, 0.125])),
    plot_contour(metric_definitions['Turbulence'], fish_sizes, tf_levels, plt.axes([0.10, 0.450, 0.35, 0.125])),
    plot_contour(metric_definitions['SlotWidth'], fish_sizes, tf_levels, plt.axes([0.10, 0.275, 0.35, 0.125])),
    plot_contour(metric_definitions['MinDepth'], fish_sizes, tf_levels, plt.axes([0.10, 0.100, 0.35, 0.125])),

    plot_metric_line(metric_definitions, fish_sizes, plt.axes([0.55, 0.100, 0.40, 0.825]))
    plt.show()

interactive(plot_metric,
   entry_velocity=widgets.FloatSlider(value=0.8, min=0.0, max=1.5, step=0.05, description='Entry Slot Velocity:', style=dict(description_width='initial')),
   max_slot_loss=widgets.FloatSlider(value=120, min=0.0, max=400.0, step=1.0, description='Max Slot Loss:', style=dict(description_width='initial')),
   max_ave_turb=widgets.FloatSlider(value=60, min=0.0, max=150.0, step=1.0, description='Max Cell Turbulence:', style=dict(description_width='initial')),
   min_slot_width=widgets.FloatSlider(value=200, min=0.0, max=500.0, step=1.0, description='Minimum Slot Width:', style=dict(description_width='initial')),
   min_depth=widgets.FloatSlider(value=1.0, min=0.0, max=3.0, step=0.05, description='Minimum Depth:', style=dict(description_width='initial')))

interactive(children=(FloatSlider(value=0.8, description='Entry Slot Velocity:', max=1.5, step=0.05, style=Sli…